In [3]:
import serial
import time
import csv
import os

PORT = 'COM3'  # Change to your Arduino's port if needed
BAUD = 9600

# Data type selection
options = [
    "Baseline Readings",
    "Soy Sauce",
    "Fish Sauce",
    "Oyster Sauce",
    "Worcestershire Sauce"
]
print("Select data type:")
for i, opt in enumerate(options, 1):
    print(f"{i}. {opt}")
choice = input("Enter number (1-5): ").strip()

try:
    choice = int(choice)
    assert 1 <= choice <= 5
except:
    print("Invalid selection. Exiting.")
    exit(1)

selected = options[choice-1]

# File and label setup
if selected == "Baseline Readings":
    filename = "baseline.csv"
    mode = "w"  # Overwrite
    trial = None
else:
    filename = selected.lower().replace(" ", "_") + ".csv"
    mode = "r+" if os.path.exists(filename) else "w+"
    trial = input("Enter trial number: ").strip()
    try:
        trial = int(trial)
        assert trial > 0
    except:
        print("Invalid trial number. Exiting.")
        exit(1)

try:
    ser = serial.Serial(PORT, BAUD, timeout=1)
    time.sleep(2)  # Wait for Arduino to reset
except Exception as e:
    print(f"Error opening serial port {PORT}: {e}")
    exit(1)

header = ["Trial", "MQ136", "MQ2", "MQ3", "MQ135", "MQ138", "MQ137", "Label"]
if selected == "Baseline Readings":
    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header[1:])
        print(f"Recording baseline readings to {filename} for 1 minute. Press Ctrl+C to stop early.")
        start_time = time.time()
        try:
            while time.time() - start_time < 60:  
                line = ser.readline().decode('utf-8', errors='ignore').strip()
                if line:
                    values = line.split(",")
                    if len(values) == 6:
                        writer.writerow(values + ["Baseline"])
                        print(values, "Baseline")
                    else:
                        print(f"Unexpected data format: {line}")
        except KeyboardInterrupt:
            print("Stopped by user.")
else:
    # Read all rows, update or append trial
    with open(filename, mode, newline="") as f:
        reader = csv.reader(f)
        rows = list(reader)
        # If file is empty, add header
        if not rows:
            rows.append(header)
        # Remove all rows for this trial
        rows = [row for row in rows if not (row and row[0] == f"Trial {trial}")]
        print(f"Recording trial {trial} for {selected} to {filename} for 10 minutes. Press Ctrl+C to stop early.")
        new_trial_rows = []
        start_time = time.time()
        try:
            while time.time() - start_time < 600:
                line = ser.readline().decode('utf-8', errors='ignore').strip()
                if line:
                    values = line.split(",")
                    if len(values) == 6:
                        new_row = [f"Trial {trial}"] + values + [selected]
                        new_trial_rows.append(new_row)
                        print(f"Trial {trial}", values, selected)
                    else:
                        print(f"Unexpected data format: {line}")
        except KeyboardInterrupt:
            print("Stopped by user.")
        # Write all rows back to file, with new trial data
        f.seek(0)
        f.truncate()
        writer = csv.writer(f)
        writer.writerows(rows + new_trial_rows)

ser.close()
print("Serial port closed.")

Select data type:
1. Baseline Readings
2. Soy Sauce
3. Fish Sauce
4. Oyster Sauce
5. Worcestershire Sauce
Recording trial 12 for Soy Sauce to soy_sauce.csv for 10 minutes. Press Ctrl+C to stop early.
Trial 12 ['40', '319', '38', '86', '507', '591'] Soy Sauce
Trial 12 ['40', '323', '40', '88', '512', '596'] Soy Sauce
Trial 12 ['41', '327', '40', '90', '516', '601'] Soy Sauce
Trial 12 ['42', '332', '42', '94', '521', '607'] Soy Sauce
Trial 12 ['43', '336', '42', '95', '525', '611'] Soy Sauce
Trial 12 ['43', '340', '44', '99', '530', '615'] Soy Sauce
Trial 12 ['43', '343', '44', '101', '535', '621'] Soy Sauce
Trial 12 ['43', '348', '45', '104', '539', '626'] Soy Sauce
Trial 12 ['44', '351', '46', '106', '544', '630'] Soy Sauce
Trial 12 ['45', '356', '47', '109', '548', '634'] Soy Sauce
Trial 12 ['46', '359', '48', '112', '552', '638'] Soy Sauce
Trial 12 ['46', '363', '49', '114', '557', '642'] Soy Sauce
Trial 12 ['46', '366', '49', '116', '561', '646'] Soy Sauce
Trial 12 ['47', '370', '50

In [ ]:
import pandas as pd

# Sensor columns
sensor_cols = ["MQ136", "MQ2", "MQ3", "MQ135", "MQ138", "MQ137"]

# Load baseline and compute averages
baseline_df = pd.read_csv("baseline.csv")
baseline_avg = baseline_df[sensor_cols].astype(float).mean()

# Sauce file names and output names
sauces = [
    "soy_sauce",
    "fish_sauce",
    "oyster_sauce",
    "worcestershire_sauce"
]

for sauce in sauces:
    input_file = f"{sauce}.csv"
    output_file = f"{sauce}_corrected.csv"
    try:
        df = pd.read_csv(input_file)
        # Subtract baseline average from each sensor column
        for col in sensor_cols:
            df[col] = df[col].astype(float) - baseline_avg[col]
        df.to_csv(output_file, index=False)
        print(f"Corrected file saved: {output_file}")
    except Exception as e:
        print(f"Error processing {input_file}: {e}")

Corrected file saved: soy_sauce_corrected.csv
Corrected file saved: fish_sauce_corrected.csv
Corrected file saved: oyster_sauce_corrected.csv
Corrected file saved: worcestershire_sauce_corrected.csv


In [21]:
# USE FOR SUMMARIZATION OF DATA

import pandas as pd

# Sensor columns in your desired order
sensor_cols = ["MQ2", "MQ3", "MQ135", "MQ136", "MQ137", "MQ138"]
sauces = [
    "soy_sauce",
    "fish_sauce",
    "oyster_sauce",
    "worcestershire_sauce"
]

summary_rows = []

for sauce in sauces:
    input_file = f"{sauce}.csv"
    try:
        df = pd.read_csv(input_file)
        # Group by trial
        for trial, group in df.groupby("Trial"):
            means = group[sensor_cols].astype(float).mean()
            row = [sauce.replace("_", " ").title()] + list(means)
            summary_rows.append(row)
    except Exception as e:
        print(f"Error processing {input_file}: {e}")

# Create summary DataFrame
summary_df = pd.DataFrame(
    summary_rows,
    columns=["Label"] + sensor_cols
)

summary_df.to_csv("sauce_trials_summary_mean.csv", index=False)
print("Mean summarized trial data saved to sauce_trials_summary_mean.csv")

Mean summarized trial data saved to sauce_trials_summary_mean.csv


In [32]:
# MEAN STD MAX MIN AUC

import pandas as pd
import numpy as np

sensor_cols = ["MQ136", "MQ2", "MQ3", "MQ135", "MQ138", "MQ137"]
sauces = [
    "soy_sauce",
    "fish_sauce",
    "oyster_sauce",
    "worcestershire_sauce"
]

summary_rows = []
feature_names = []
for sensor in sensor_cols:
    feature_names += [f"{sensor}_mean", f"{sensor}_std", f"{sensor}_max", f"{sensor}_min", f"{sensor}_auc"]

for sauce in sauces:
    input_file = f"{sauce}.csv"
    try:
        df = pd.read_csv(input_file)
        # Group by trial
        for trial, group in df.groupby("Trial"):
            features = []
            for sensor in sensor_cols:
                vals = group[sensor].astype(float).values
                features += [
                    np.mean(vals),
                    np.std(vals),
                    np.max(vals),
                    np.min(vals),
                    np.sum(vals)  # AUC as sum
                ]
            row = [trial] + features + [sauce.replace("_", " ").title()]
            summary_rows.append(row)
    except Exception as e:
        print(f"Error processing {input_file}: {e}")

# Create summary DataFrame
summary_df = pd.DataFrame(
    summary_rows,
    columns=["Trial"] + feature_names + ["Label"]
)

summary_df.to_csv("sauce_trials_summary_advanced.csv", index=False)
print("Advanced summarized trial data saved to sauce_trials_summary_advanced.csv")

Advanced summarized trial data saved to sauce_trials_summary_advanced.csv
